# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [ ]:
# %pip install -r requirements.txt

In [ ]:
import os
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
import math
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
# from imblearn.under_sampling import RandomUnderSampler


# if 'PG_PORT' in os.environ:

#     host = 'db'
#     port = os.environ['PG_PORT']
    # database = os.environ['PG_DATABASE']
#     user = os.environ['PG_USER']
#     password = os.environ['PG_PASSWORD']
    
#     # pd.read_sql用のコネクタ
#     conn = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")

#     df_customer = pd.read_sql(sql='select * from customer', con=conn)
#     df_category = pd.read_sql(sql='select * from category', con=conn)
#     df_product = pd.read_sql(sql='select * from product', con=conn)
#     df_receipt = pd.read_sql(sql='select * from receipt', con=conn)
#     df_store = pd.read_sql(sql='select * from store', con=conn)
#     df_geocode = pd.read_sql(sql='select * from geocode', con=conn)

# else:
#     if not os.path.exists('../data/'):
#         !git clone https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess
#         os.chdir('100knocks-preprocess/docker/work/answer')

dtype = {
    'customer_id': str,
    'gender_cd': str,
    'postal_cd': str,
    'application_store_cd': str,
    'status_cd': str,
    'category_major_cd': str,
    'category_medium_cd': str,
    'category_small_cd': str,
    'product_cd': str,
    'store_cd': str,
    'prefecture_cd': str,
    'tel_no': str,
    'postal_cd': str,
    'street': str
}

df_customer = pd.read_csv("../data/customer.csv", dtype=dtype)
df_category = pd.read_csv("../data/category.csv", dtype=dtype)
df_product = pd.read_csv("../data/product.csv", dtype=dtype)
df_receipt = pd.read_csv("../data/receipt.csv", dtype=dtype)
df_store = pd.read_csv("../data/store.csv", dtype=dtype)
df_geocode = pd.read_csv("../data/geocode.csv", dtype=dtype)


# 演習問題

---
> P-001: レシート明細データ（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [ ]:
# df_receiptの先頭10行を表示して、データの内容を目視で確認します。
# .head(10)メソッドは、DataFrameの最初の10行を返します。
df_receipt.head(10)

---
> P-002: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。

In [ ]:
# df_receiptから指定された列（sales_ymd, customer_id, product_cd, amount）を抽出し、
# その先頭10行を表示します。
# 列の選択は二重の角括弧（[['col1', 'col2']]）で行います。
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].head(10)

---
> P-003: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。ただし、sales_ymdをsales_dateに項目名を変更して抽出すること。

In [ ]:
# df_receiptから指定された列を抽出し、sales_ymd列をsales_dateにリネームします。
# その後、先頭10行を表示します。
# .rename(columns={'old_name': 'new_name'})で列名を変更できます。
df_receipt[['sales_ymd', 'customer_id', 'product_cd', 'amount']].rename(columns={'sales_ymd': 'sales_date'}).head(10)

---
> P-004: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [ ]:
# df_receiptから指定された列を抽出し、customer_idが"CS018205000001"の行をフィルタリングします。
# .query()メソッドを使うと、文字列で条件式を記述できます。
df_receipt.query('customer_id == "CS018205000001"')[['sales_ymd', 'customer_id', 'product_cd', 'amount']]

---
> P-005: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [ ]:
# df_receiptから指定された列を抽出し、customer_idが"CS018205000001"かつamountが1000以上の行をフィルタリングします。
# 複数の条件はandで結合します。
df_receipt.query('customer_id == "CS018205000001" and amount >= 1000')[['sales_ymd', 'customer_id', 'product_cd', 'amount']]

---
> P-006: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [ ]:
# df_receiptから指定された列を抽出し、customer_idが"CS018205000001"かつ
# (amountが1000以上 または quantityが5以上)の行をフィルタリングします。
# OR条件は()で囲み、orで結合します。
df_receipt.query('customer_id == "CS018205000001" and (amount >= 1000 or quantity >= 5)')[['sales_ymd', 'customer_id', 'product_cd', 'quantity', 'amount']]

---
> P-007: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [ ]:
# df_receiptから指定された列を抽出し、customer_idが"CS018205000001"かつ
# amountが1000以上2000以下の行をフィルタリングします。
# 範囲指定は1000 <= amount <= 2000のように記述できます。
df_receipt.query('customer_id == "CS018205000001" and 1000 <= amount <= 2000')[['sales_ymd', 'customer_id', 'product_cd', 'amount']]

---
> P-008: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [ ]:
# df_receiptから指定された列を抽出し、customer_idが"CS018205000001"かつ
# product_cdが"P071401019"ではない行をフィルタリングします。
# 否定条件は!=で記述します。
df_receipt.query('customer_id == "CS018205000001" and product_cd != "P071401019"')[['sales_ymd', 'customer_id', 'product_cd', 'amount']]

---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。
> 
> `df_store.query('not(prefecture_cd == "13" | floor_area > 900)')`

In [ ]:
# ド・モルガンの法則を適用して、OR条件をAND条件に書き換えます。
# not (A or B) は (not A) and (not B) と等価です。
# したがって、not(prefecture_cd == "13" | floor_area > 900) は
# prefecture_cd != "13" and floor_area <= 900 となります。
df_store.query('prefecture_cd != "13" and floor_area <= 900')

---
> P-010: 店舗データ（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件表示せよ。

In [ ]:
# df_storeからstore_cdが"S14"で始まる行をフィルタリングし、先頭10行を表示します。
# .str.startswith()メソッドを使用して、文字列の先頭が特定の文字列であるかを判定します。
df_store[df_store['store_cd'].str.startswith('S14')].head(10)

---
> P-011: 顧客データ（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件表示せよ。

In [ ]:
# df_customerからcustomer_idの末尾が'1'の行をフィルタリングし、先頭10行を表示します。
# .str.endswith()メソッドを使用して、文字列の末尾が特定の文字列であるかを判定します。
df_customer[df_customer['customer_id'].str.endswith('1')].head(10)

---
> P-012: 店舗データ（df_store）から、住所 (address) に"横浜市"が含まれるものだけ全項目表示せよ。

In [ ]:
# df_storeからaddress列に"横浜市"が含まれる行をフィルタリングします。
# .str.contains()メソッドを使用して、文字列に特定のサブストリングが含まれるかを判定します。
df_store[df_store['address'].str.contains('横浜市')]

---
> P-013: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件表示せよ。

In [ ]:
# df_customerからstatus_cdの先頭がAからFのいずれかで始まる行をフィルタリングし、先頭10行を表示します。
# .str.match()メソッドと正規表現を使用します。
# ^[A-F] は文字列の先頭がAからFのいずれかであることを意味します。
df_customer[df_customer['status_cd'].str.match(r'^[A-F]')].head(10)

---
> P-014: 顧客データ（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [ ]:
# df_customerからstatus_cdの末尾が1から9のいずれかで終わる行をフィルタリングし、先頭10行を表示します。
# .str.match()メソッドと正規表現を使用します。
# .*[1-9]$ は任意の文字が0回以上続き、末尾が1から9のいずれかであることを意味します。
df_customer[df_customer['status_cd'].str.match(r'.*[1-9]$')].head(10)

---
> P-015: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [ ]:
# df_customerからstatus_cdの先頭がAからFのいずれかで始まり、
# かつ末尾が1から9のいずれかで終わる行をフィルタリングし、先頭10行を表示します。
# 正規表現 ^[A-F].*[1-9]$ を使用します。
# ^[A-F] は先頭がA-F、.*は任意の文字が0回以上、[1-9]$ は末尾が1-9であることを意味します。
df_customer[df_customer['status_cd'].str.match(r'^[A-F].*[1-9]$')].head(10)

---
> P-016: 店舗データ（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [ ]:
# df_storeからtel_noがXXX-XXX-XXXXの形式にマッチする行をフィルタリングします。
# 正規表現 ^\d{3}-\d{3}-\d{4}$ を使用します。
# ^ は文字列の先頭、\d{n} は数字がn回繰り返されること、- はハイフン、
# $ は文字列の末尾を意味します。
df_store[df_store['tel_no'].str.match(r'^\d{3}-\d{3}-\d{4}$')]

---
> P-017: 顧客データ（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭から全項目を10件表示せよ。

In [ ]:
# df_customerをbirth_dayで昇順（ascending=True）にソートします。
# 生年月日は若いほど日付が新しく、高齢ほど日付が古くなるため、昇順でソートすると高齢順になります。
# その後、先頭10行を表示します。
df_customer.sort_values(by='birth_day', ascending=True).head(10)

---
> P-018: 顧客データ（df_customer）を生年月日（birth_day）で若い順にソートし、先頭から全項目を10件表示せよ。

In [ ]:
# df_customerを生年月日（birth_day）で降順（ascending=False）にソートします。
# 生年月日は若いほど日付が新しく、降順でソートすると若い順になります。
# その後、先頭10行を表示します。
df_customer.sort_values(by='birth_day', ascending=False).head(10)

---
> P-019: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [ ]:
# df_receiptのamount列に対し、高い順にランクを付与します。
# method='min'を指定することで、同じ値には同じランクの最小値を付与します。
# その後、customer_id, amount, ranking列を抽出し、rankingでソートして先頭10行を表示します。
df_receipt['ranking'] = df_receipt['amount'].rank(method='min', ascending=False)
df_receipt[['customer_id', 'amount', 'ranking']].sort_values('ranking').head(10)

---
> P-020: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [ ]:
# df_receiptのamount列に対し、高い順にランクを付与します。
# method='first'を指定することで、同じ値でも出現順に異なるランクを付与します。
# その後、customer_id, amount, ranking列を抽出し、rankingでソートして先頭10行を表示します。
df_receipt['ranking'] = df_receipt['amount'].rank(method='first', ascending=False)
df_receipt[['customer_id', 'amount', 'ranking']].sort_values('ranking').head(10)

---
> P-021: レシート明細データ（df_receipt）に対し、件数をカウントせよ。

In [ ]:
# df_receiptの行数（レコード数）をカウントします。
# len()関数を使用するか、.shape[0]でDataFrameの行数を取得できます。
len(df_receipt)

---
> P-022: レシート明細データ（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [ ]:
# df_receiptのcustomer_id列のユニークな値の数をカウントします。
# .nunique()メソッドを使用します。
df_receipt['customer_id'].nunique()

---
> P-023: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [ ]:
# df_receiptをstore_cdでグループ化し、amountとquantityの合計を計算します。
# .groupby()と.sum()を組み合わせて使用します。
df_receipt.groupby('store_cd')[['amount', 'quantity']].sum()

---
> P-024: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）を求め、10件表示せよ。

In [ ]:
# df_receiptをcustomer_idでグループ化し、sales_ymdの最大値（最も新しい年月日）を求めます。
# その後、先頭10行を表示します。
df_receipt.groupby('customer_id')['sales_ymd'].max().head(10)

---
> P-025: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上年月日（sales_ymd）を求め、10件表示せよ。

In [ ]:
# df_receiptをcustomer_idでグループ化し、sales_ymdの最小値（最も古い年月日）を求めます。
# その後、先頭10行を表示します。
df_receipt.groupby('customer_id')['sales_ymd'].min().head(10)

---
> P-026: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）と古い売上年月日を求め、両者が異なるデータを10件表示せよ。

In [ ]:
# df_receiptをcustomer_idでグループ化し、sales_ymdの最小値と最大値を同時に求めます。
# .agg(['min', 'max'])を使用します。
# その後、最小値と最大値が異なる（つまり、複数回購入している）顧客をフィルタリングし、先頭10行を表示します。
df_sales_ymd = df_receipt.groupby('customer_id')['sales_ymd'].agg(['min', 'max'])
df_sales_ymd[df_sales_ymd['min'] != df_sales_ymd['max']].head(10)

---
> P-027: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [ ]:
# df_receiptをstore_cdでグループ化し、amountの平均を計算します。
# その結果を降順にソートし、先頭5行を表示します。
df_receipt.groupby('store_cd')['amount'].mean().sort_values(ascending=False).head(5)

---
> P-028: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [ ]:
# df_receiptをstore_cdでグループ化し、amountの中央値を計算します。
# その結果を降順にソートし、先頭5行を表示します。
df_receipt.groupby('store_cd')['amount'].median().sort_values(ascending=False).head(5)

---
> P-029: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求め、10件表示させよ。

In [ ]:
# df_receiptをstore_cdでグループ化し、product_cdの最頻値を求めます。
# .mode()はSeriesを返すため、[0]で最初の最頻値を取得します。
# その後、先頭10行を表示します。
df_receipt.groupby('store_cd')['product_cd'].apply(lambda x: x.mode()[0]).head(10)

---
> P-030: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の分散を計算し、降順で5件表示せよ。

In [ ]:
# df_receiptをstore_cdでグループ化し、amountの分散を計算します。
# その結果を降順にソートし、先頭5行を表示します。
# Pandasの.var()メソッドはデフォルトで不偏分散（ddof=1）を計算します。
df_receipt.groupby('store_cd')['amount'].var().sort_values(ascending=False).head(5)

---
> P-031: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標準偏差を計算し、降順で5件表示せよ。

TIPS:

PandasとNumpyでddofのデフォルト値が異なることに注意しましょう
```
Pandas：
DataFrame.std(self, axis=None, skipna=None, level=None, ddof=1, numeric_only=None, **kwargs)
Numpy:
numpy.std(a, axis=None, dtype=None, out=None, ddof=0, keepdims=)
```

In [ ]:
# df_receiptをstore_cdでグループ化し、amountの標準偏差を計算します。
# その結果を降順にソートし、先頭5行を表示します。
# Pandasの.std()メソッドはデフォルトで不偏標準偏差（ddof=1）を計算します。
df_receipt.groupby('store_cd')['amount'].std().sort_values(ascending=False).head(5)

---
> P-032: レシート明細データ（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [ ]:
# df_receiptのamount列について、25%, 50%, 75%, 100%のパーセンタイル値を計算します。
# .quantile()メソッドを使用し、引数にパーセンタイル値をリストで渡します。
df_receipt['amount'].quantile([0.25, 0.5, 0.75, 1.0])

---
> P-033: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [ ]:
# df_receiptをstore_cdでグループ化し、amountの平均を計算します。
# その結果から、平均が330以上のものをフィルタリングします。
df_receipt.groupby('store_cd')['amount'].mean()[df_receipt.groupby('store_cd')['amount'].mean() >= 330]

---
> P-034: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [ ]:
# df_receiptからcustomer_idが"Z"で始まらない行をフィルタリングします。
# その後、customer_idごとにamountを合計し、その合計値の平均を計算します。
df_receipt_filtered = df_receipt[~df_receipt['customer_id'].str.startswith('Z')]
df_receipt_filtered.groupby('customer_id')['amount'].sum().mean()

---
> P-035: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出し、10件表示せよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [ ]:
# df_receiptからcustomer_idが"Z"で始まらない行をフィルタリングします。
# その後、customer_idごとにamountを合計します。
# 全顧客の平均売上金額を計算し、平均以上に買い物をしている顧客をフィルタリングして先頭10行を表示します。
df_receipt_filtered = df_receipt[~df_receipt['customer_id'].str.startswith('Z')]
customer_amount_sum = df_receipt_filtered.groupby('customer_id')['amount'].sum()
avg_amount = customer_amount_sum.mean()
customer_amount_sum[customer_amount_sum >= avg_amount].head(10)

---
> P-036: レシート明細データ（df_receipt）と店舗データ（df_store）を内部結合し、レシート明細データの全項目と店舗データの店舗名（store_name）を10件表示せよ。

In [ ]:
# df_receiptとdf_storeを内部結合します。
# 共通のキーは'store_cd'です。
# その後、レシート明細データの全項目と店舗データのstore_nameを抽出し、先頭10行を表示します。
df_merged = pd.merge(df_receipt, df_store, on='store_cd', how='inner')
df_merged.head(10)

---
> P-037: 商品データ（df_product）とカテゴリデータ（df_category）を内部結合し、商品データの全項目とカテゴリデータのカテゴリ小区分名（category_small_name）を10件表示せよ。

In [ ]:
# df_productとdf_categoryを内部結合します。
# 共通のキーは'category_small_cd'です。
# その後、商品データの全項目とカテゴリデータのcategory_small_nameを抽出し、先頭10行を表示します。
df_merged = pd.merge(df_product, df_category, on='category_small_cd', how='inner')
df_merged.head(10)

---
> P-038: 顧客データ（df_customer）とレシート明細データ（df_receipt）から、顧客ごとの売上金額合計を求め、10件表示せよ。ただし、売上実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [ ]:
# df_customerから性別コードが女性(1)で、かつ顧客IDが"Z"で始まらない顧客をフィルタリングします。
# df_receiptからcustomer_idごとにamountを合計します。
# フィルタリングした顧客データと売上金額合計を外部結合し、売上実績がない顧客の売上金額を0で埋めます。
# その後、先頭10行を表示します。
df_customer_filtered = df_customer.query('gender_cd == "1" and not customer_id.str.startswith("Z")')
df_receipt_sum = df_receipt.groupby('customer_id')['amount'].sum().reset_index()
df_merged = pd.merge(df_customer_filtered[['customer_id']], df_receipt_sum, on='customer_id', how='left').fillna(0)
df_merged.head(10)

---
> P-039: レシート明細データ（df_receipt）から、売上日数の多い顧客の上位20件を抽出したデータと、売上金額合計の多い顧客の上位20件を抽出したデータをそれぞれ作成し、さらにその2つを完全外部結合せよ。ただし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [ ]:
# 非会員を除外
df_receipt_member = df_receipt[~df_receipt['customer_id'].str.startswith('Z')]

# 売上日数の多い顧客の上位20件
# customer_idごとにsales_ymdのユニークな数をカウントし、降順にソートして上位20件を抽出
top20_sales_days = df_receipt_member.groupby('customer_id')['sales_ymd'].nunique().nlargest(20).reset_index()
top20_sales_days.columns = ['customer_id', 'sales_days']

# 売上金額合計の多い顧客の上位20件
# customer_idごとにamountを合計し、降順にソートして上位20件を抽出
top20_sales_amount = df_receipt_member.groupby('customer_id')['amount'].sum().nlargest(20).reset_index()
top20_sales_amount.columns = ['customer_id', 'total_amount']

# 2つのデータを完全外部結合
# how='outer'で完全外部結合を行います。
df_merged = pd.merge(top20_sales_days, top20_sales_amount, on='customer_id', how='outer')
df_merged

---
> P-040: 全ての店舗と全ての商品を組み合わせたデータを作成したい。店舗データ（df_store）と商品データ（df_product）を直積し、件数を計算せよ。

In [ ]:
# df_storeとdf_productの直積（デカルト積）を計算します。
# これを行うには、両方のDataFrameにダミーのキー列を追加し、そのキーで結合します。
# その後、結果の行数（件数）を計算します。
df_store['dummy_key'] = 1
df_product['dummy_key'] = 1
df_cross_join = pd.merge(df_store, df_product, on='dummy_key').drop('dummy_key', axis=1)
len(df_cross_join)

---
> P-041: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前回売上があった日からの売上金額増減を計算せよ。そして結果を10件表示せよ。

In [ ]:
# sales_ymdを日付型に変換します。
df_receipt['sales_ymd'] = pd.to_datetime(df_receipt['sales_ymd'].astype(str))

# sales_ymdごとにamountを合計します。
df_daily_sales = df_receipt.groupby('sales_ymd')['amount'].sum().reset_index()

# 前回売上があった日からの売上金額増減を計算します。
# .diff()メソッドで差分を計算します。
df_daily_sales['amount_diff'] = df_daily_sales['amount'].diff()
df_daily_sales.head(10)

---
> P-042: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、前回、前々回、3回前に売上があった日のデータを結合せよ。そして結果を10件表示せよ。

In [ ]:
# sales_ymdを日付型に変換します。
df_receipt['sales_ymd'] = pd.to_datetime(df_receipt['sales_ymd'].astype(str))

# sales_ymdごとにamountを合計します。
df_daily_sales = df_receipt.groupby('sales_ymd')['amount'].sum().reset_index()

# 前回、前々回、3回前の売上データを結合します。
# .shift()メソッドで指定した数だけ行をずらします。
df_daily_sales['amount_prev1'] = df_daily_sales['amount'].shift(1)
df_daily_sales['amount_prev2'] = df_daily_sales['amount'].shift(2)
df_daily_sales['amount_prev3'] = df_daily_sales['amount'].shift(3)
df_daily_sales.head(10)

---
> P-043： レシート明細データ（df_receipt）と顧客データ（df_customer）を結合し、性別コード（gender_cd）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータを作成せよ。性別コードは0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [ ]:
# df_receiptとdf_customerをcustomer_idで結合します。
df_merged = pd.merge(df_receipt, df_customer, on='customer_id', how='inner')

# 年齢から年代を計算します。
# 10歳ごとの階級とし、60歳以上は全て60歳代とします。
def get_age_band(age):
    if age >= 60:
        return '60s'
    else:
        return f'{age // 10 * 10}s'

df_merged['age_band'] = df_merged['age'].apply(get_age_band)

# 性別コードを文字列に変換します。
df_merged['gender_cd'] = df_merged['gender_cd'].replace({'0': '男性', '1': '女性', '9': '不明'})

# 性別コードと年代ごとに売上金額を合計し、クロス集計します。
df_sales_summary = pd.pivot_table(df_merged,
                                  index='age_band',
                                  columns='gender_cd',
                                  values='amount',
                                  aggfunc='sum',
                                  fill_value=0)

# 列の順番を年代、女性、男性、性別不明にします。
df_sales_summary = df_sales_summary[['女性', '男性', '不明']]
df_sales_summary

---
> P-044： 043で作成した売上サマリデータ（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を"00"、女性を"01"、不明を"99"とする。

In [ ]:
# df_sales_summaryを縦持ちに変換します。
# .melt()メソッドを使用します。
# id_varsで年代（age_band）を固定し、value_varsで性別の列を指定します。
# var_nameで新しい性別コードの列名、value_nameで売上金額の列名を指定します。
df_sales_summary_melted = df_sales_summary.melt(id_vars='age_band',
                                                value_vars=['女性', '男性', '不明'],
                                                var_name='gender_cd',
                                                value_name='amount')

# 性別コードを問題の指示に従って変換します。
df_sales_summary_melted['gender_cd'] = df_sales_summary_melted['gender_cd'].replace({'男性': '00', '女性': '01', '不明': '99'})
df_sales_summary_melted

---
> P-045: 顧客データ（df_customer）の生年月日（birth_day）は日付型でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [ ]:
# df_customerのbirth_day列を日付型に変換します。
# その後、strftime('%Y%m%d')を使用してYYYYMMDD形式の文字列に変換します。
# customer_idとともに先頭10行を表示します。
df_customer['birth_day'] = pd.to_datetime(df_customer['birth_day'])
df_customer['birth_day_str'] = df_customer['birth_day'].dt.strftime('%Y%m%d')
df_customer[['customer_id', 'birth_day_str']].head(10)

---
> P-046: 顧客データ（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [ ]:
# df_customerのapplication_date列を日付型に変換します。
# pd.to_datetime()関数を使用します。
# customer_idとともに先頭10行を表示します。
df_customer['application_date'] = pd.to_datetime(df_customer['application_date'])
df_customer[['customer_id', 'application_date']].head(10)

---
> P-047: レシート明細データ（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型に変換し、レシート番号（receipt_no）、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [ ]:
# df_receiptのsales_ymd列を日付型に変換します。
# 数値型なので、まず文字列に変換してからpd.to_datetime()を使用します。
# receipt_no, receipt_sub_noとともに先頭10行を表示します。
df_receipt['sales_ymd'] = pd.to_datetime(df_receipt['sales_ymd'].astype(str))
df_receipt[['receipt_no', 'receipt_sub_no', 'sales_ymd']].head(10)

---
> P-048: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [ ]:
# df_receiptのsales_epoch列を日付型に変換します。
# pd.to_datetime()関数にunit='s'を指定することでUNIX秒を日付に変換できます。
# receipt_no, receipt_sub_noとともに先頭10行を表示します。
df_receipt['sales_epoch_datetime'] = pd.to_datetime(df_receipt['sales_epoch'], unit='s')
df_receipt[['receipt_no', 'receipt_sub_no', 'sales_epoch_datetime']].head(10)

---
> P-049: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「年」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [ ]:
# df_receiptのsales_epoch列を日付型に変換します。
# .dt.yearで年だけを取り出します。
# receipt_no, receipt_sub_noとともに先頭10行を表示します。
df_receipt['sales_epoch_datetime'] = pd.to_datetime(df_receipt['sales_epoch'], unit='s')
df_receipt['sales_year'] = df_receipt['sales_epoch_datetime'].dt.year
df_receipt[['receipt_no', 'receipt_sub_no', 'sales_year']].head(10)

---
> P-050: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「月」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「月」は0埋め2桁で取り出すこと。

In [ ]:
# df_receiptのsales_epoch列を日付型に変換します。
# .dt.strftime('%m')で月を0埋め2桁で取り出します。
# receipt_no, receipt_sub_noとともに先頭10行を表示します。
df_receipt['sales_epoch_datetime'] = pd.to_datetime(df_receipt['sales_epoch'], unit='s')
df_receipt['sales_month'] = df_receipt['sales_epoch_datetime'].dt.strftime('%m')
df_receipt[['receipt_no', 'receipt_sub_no', 'sales_month']].head(10)

---
> P-051: レシート明細データ（df_receipt）の売上エポック秒を日付型に変換し、「日」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「日」は0埋め2桁で取り出すこと。

In [ ]:
# df_receiptのsales_epoch列を日付型に変換します。
# .dt.strftime('%d')で日を0埋め2桁で取り出します。
# receipt_no, receipt_sub_noとともに先頭10行を表示します。
df_receipt['sales_epoch_datetime'] = pd.to_datetime(df_receipt['sales_epoch'], unit='s')
df_receipt['sales_day'] = df_receipt['sales_epoch_datetime'].dt.strftime('%d')
df_receipt[['receipt_no', 'receipt_sub_no', 'sales_day']].head(10)

---
> P-052: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2,000円以下を0、2,000円より大きい金額を1に二値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
# df_receiptからcustomer_idが"Z"で始まらない行をフィルタリングします。
# customer_idごとにamountを合計します。
df_receipt_filtered = df_receipt[~df_receipt['customer_id'].str.startswith('Z')]
customer_amount_sum = df_receipt_filtered.groupby('customer_id')['amount'].sum().reset_index()

# 売上金額合計を二値化します。
# 2000円以下は0、2000円より大きい場合は1とします。
customer_amount_sum['amount_binary'] = (customer_amount_sum['amount'] > 2000).astype(int)
customer_amount_sum.head(10)

---
> P-053: 顧客データ（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に二値化せよ。さらにレシート明細データ（df_receipt）と結合し、全期間において売上実績のある顧客数を、作成した二値ごとにカウントせよ。

In [ ]:
# df_customerのpostal_cdを二値化します。
# 先頭3桁が100から209の範囲にあるかを判定します。
df_customer['postal_cd_binary'] = df_customer['postal_cd'].apply(lambda x: 1 if 100 <= int(x[:3]) <= 209 else 0)

# df_receiptから売上実績のあるcustomer_idを抽出します。
# df_customerと売上実績のあるcustomer_idを結合し、売上実績のある顧客のみをフィルタリングします。
df_receipt_customers = df_receipt['customer_id'].unique()
df_customer_sales = df_customer[df_customer['customer_id'].isin(df_receipt_customers)]

# 二値化したpostal_cd_binaryごとに売上実績のある顧客数をカウントします。
df_customer_sales.groupby('postal_cd_binary')['customer_id'].count()

---
> P-054: 顧客データ（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに10件表示せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。

In [ ]:
# 住所の都道府県名に基づいてコード値を割り当てます。
# .apply()とラムダ関数、または.map()を使用します。
def get_prefecture_code(address):
    if '埼玉県' in address:
        return 11
    elif '千葉県' in address:
        return 12
    elif '東京都' in address:
        return 13
    elif '神奈川県' in address:
        return 14
    else:
        return None # または適切なデフォルト値

df_customer['prefecture_cd'] = df_customer['address'].apply(get_prefecture_code)
df_customer[['customer_id', 'address', 'prefecture_cd']].head(10)

---
> P-055: レシート明細（df_receipt）データの売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額合計とともに10件表示せよ。カテゴリ値は順に1〜4とする。
>
> - 最小値以上第1四分位未満 ・・・ 1を付与
> - 第1四分位以上第2四分位未満 ・・・ 2を付与
> - 第2四分位以上第3四分位未満 ・・・ 3を付与
> - 第3四分位以上 ・・・ 4を付与

In [ ]:
# customer_idごとにamountを合計します。
customer_amount_sum = df_receipt.groupby('customer_id')['amount'].sum().reset_index()

# 合計金額の四分位点を計算します。
quantiles = customer_amount_sum['amount'].quantile([0.25, 0.5, 0.75])
q1 = quantiles[0.25]
q2 = quantiles[0.5]
q3 = quantiles[0.75]

# カテゴリ値を作成します。
def create_category(amount):
    if amount < q1:
        return 1
    elif q1 <= amount < q2:
        return 2
    elif q2 <= amount < q3:
        return 3
    else:
        return 4

customer_amount_sum['amount_category'] = customer_amount_sum['amount'].apply(create_category)
customer_amount_sum[['customer_id', 'amount', 'amount_category']].head(10)

---
> P-056: 顧客データ（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに10件表示せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。

In [ ]:
# 年齢から年代を計算します。
# 10歳ごとの階級とし、60歳以上は全て60歳代とします。
def get_age_band(age):
    if age >= 60:
        return '60s'
    else:
        return f'{age // 10 * 10}s'

df_customer['age_band'] = df_customer['age'].apply(get_age_band)
df_customer[['customer_id', 'birth_day', 'age_band']].head(10)

---
> P-057: 056の抽出結果と性別コード（gender_cd）により、新たに性別×年代の組み合わせを表すカテゴリデータを作成し、10件表示せよ。組み合わせを表すカテゴリの値は任意とする。

In [ ]:
# P-056で作成したage_bandとgender_cdを組み合わせて新しいカテゴリを作成します。
# gender_cdは0:男性, 1:女性, 9:不明なので、これらを文字列に変換して結合します。
df_customer['gender_age_band'] = df_customer['gender_cd'].astype(str) + '_' + df_customer['age_band']
df_customer[['customer_id', 'gender_cd', 'age_band', 'gender_age_band']].head(10)

---
> P-058: 顧客データ（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに10件表示せよ。

In [ ]:
# df_customerのgender_cd列をダミー変数化します。
# pd.get_dummies()関数を使用します。
# customer_idとともに先頭10行を表示します。
df_gender_dummies = pd.get_dummies(df_customer['gender_cd'], prefix='gender')
df_customer_with_dummies = pd.concat([df_customer['customer_id'], df_gender_dummies], axis=1)
df_customer_with_dummies.head(10)

---
> P-059: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに10件表示せよ。標準化に使用する標準偏差は、分散の平方根、もしくは不偏分散の平方根のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

TIPS:
- query()の引数engineで'python'か'numexpr'かを選択でき、デフォルトはインストールされていればnumexprが、無ければpythonが使われます。さらに、文字列メソッドはengine='python'でないとquery()内で使えません。


In [ ]:
# df_receiptからcustomer_idが"Z"で始まらない行をフィルタリングします。
# customer_idごとにamountを合計します。
df_receipt_filtered = df_receipt[~df_receipt['customer_id'].str.startswith('Z')]
customer_amount_sum = df_receipt_filtered.groupby('customer_id')['amount'].sum().reset_index()

# 売上金額合計を標準化します。
# (X - mean) / std を計算します。
mean_amount = customer_amount_sum['amount'].mean()
std_amount = customer_amount_sum['amount'].std() # デフォルトで不偏標準偏差

customer_amount_sum['amount_standardized'] = (customer_amount_sum['amount'] - mean_amount) / std_amount
customer_amount_sum[['customer_id', 'amount', 'amount_standardized']].head(10)

---
> P-060: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
# df_receiptからcustomer_idが"Z"で始まらない行をフィルタリングします。
# customer_idごとにamountを合計します。
df_receipt_filtered = df_receipt[~df_receipt['customer_id'].str.startswith('Z')]
customer_amount_sum = df_receipt_filtered.groupby('customer_id')['amount'].sum().reset_index()

# 売上金額合計を最小値0、最大値1に正規化します。
# (X - min) / (max - min) を計算します。
min_amount = customer_amount_sum['amount'].min()
max_amount = customer_amount_sum['amount'].max()

customer_amount_sum['amount_normalized'] = (customer_amount_sum['amount'] - min_amount) / (max_amount - min_amount)
customer_amount_sum[['customer_id', 'amount', 'amount_normalized']].head(10)

---
> P-061: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を常用対数化（底10）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
# df_receiptからcustomer_idが"Z"で始まらない行をフィルタリングします。
# customer_idごとにamountを合計します。
df_receipt_filtered = df_receipt[~df_receipt['customer_id'].str.startswith('Z')]
customer_amount_sum = df_receipt_filtered.groupby('customer_id')['amount'].sum().reset_index()

# 売上金額合計を常用対数化（底10）します。
# numpyのlog10関数を使用します。
customer_amount_sum['amount_log10'] = np.log10(customer_amount_sum['amount'])
customer_amount_sum[['customer_id', 'amount', 'amount_log10']].head(10)

---
> P-062: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を自然対数化（底e）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [ ]:
# df_receiptからcustomer_idが"Z"で始まらない行をフィルタリングします。
# customer_idごとにamountを合計します。
df_receipt_filtered = df_receipt[~df_receipt['customer_id'].str.startswith('Z')]
customer_amount_sum = df_receipt_filtered.groupby('customer_id')['amount'].sum().reset_index()

# 売上金額合計を自然対数化（底e）します。
# numpyのlog関数を使用します。
customer_amount_sum['amount_log'] = np.log(customer_amount_sum['amount'])
customer_amount_sum[['customer_id', 'amount', 'amount_log']].head(10)

---
> P-063: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を算出し、結果を10件表示せよ。

In [ ]:
# df_productのunit_priceからunit_costを引いて利益額を計算します。
# その後、先頭10行を表示します。
df_product['profit'] = df_product['unit_price'] - df_product['unit_cost']
df_product[['product_cd', 'unit_price', 'unit_cost', 'profit']].head(10)

---
> P-064: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。ただし、単価と原価には欠損が生じていることに注意せよ。

---
> P-065: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [ ]:
# 利益率30%となる新たな単価を計算します。
# 利益率 = (新単価 - 原価) / 新単価
# 0.3 = (新単価 - 原価) / 新単価
# 0.3 * 新単価 = 新単価 - 原価
# 原価 = 新単価 - 0.3 * 新単価
# 原価 = 新単価 * (1 - 0.3)
# 新単価 = 原価 / (1 - 0.3) = 原価 / 0.7
# 1円未満は切り捨てるため、np.floor()を使用します。
# 欠損値がある場合はNaNになるため、計算前にdropna()で欠損値を除外します。
df_product['new_unit_price_floor'] = np.floor(df_product['unit_cost'] / 0.7)

# 新たな単価での利益率を計算し、確認します。
df_product['new_profit_rate_floor'] = (df_product['new_unit_price_floor'] - df_product['unit_cost']) / df_product['new_unit_price_floor']

# 結果を10件表示します。
df_product[['product_cd', 'unit_price', 'unit_cost', 'new_unit_price_floor', 'new_profit_rate_floor']].head(10)

---
> P-066: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を丸めること（四捨五入または偶数への丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [ ]:
# 利益率30%となる新たな単価を計算します。
# 新単価 = 原価 / 0.7
# 1円未満を丸めるため、np.round()を使用します。
# 欠損値がある場合はNaNになるため、計算前にdropna()で欠損値を除外します。
df_product['new_unit_price_round'] = np.round(df_product['unit_cost'] / 0.7)

# 新たな単価での利益率を計算し、確認します。
df_product['new_profit_rate_round'] = (df_product['new_unit_price_round'] - df_product['unit_cost']) / df_product['new_unit_price_round']

# 結果を10件表示します。
df_product[['product_cd', 'unit_price', 'unit_cost', 'new_unit_price_round', 'new_profit_rate_round']].head(10)

---
> P-067: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [ ]:
# 利益率30%となる新たな単価を計算します。
# 新単価 = 原価 / 0.7
# 1円未満を切り上げるため、np.ceil()を使用します。
# 欠損値がある場合はNaNになるため、計算前にdropna()で欠損値を除外します。
df_product['new_unit_price_ceil'] = np.ceil(df_product['unit_cost'] / 0.7)

# 新たな単価での利益率を計算し、確認します。
df_product['new_profit_rate_ceil'] = (df_product['new_unit_price_ceil'] - df_product['unit_cost']) / df_product['new_unit_price_ceil']

# 結果を10件表示します。
df_product[['product_cd', 'unit_price', 'unit_cost', 'new_unit_price_ceil', 'new_profit_rate_ceil']].head(10)

---
> P-068: 商品データ（df_product）の各商品について、消費税率10％の税込み金額を求めよ。1円未満の端数は切り捨てとし、結果を10件表示せよ。ただし、単価（unit_price）には欠損が生じていることに注意せよ。

In [ ]:
# 税込み金額を計算します。
# 税込み金額 = 単価 * 1.10
# 1円未満は切り捨てるため、np.floor()を使用します。
# 欠損値がある場合はNaNになるため、計算前にdropna()で欠損値を除外します。
df_product['price_with_tax'] = np.floor(df_product['unit_price'] * 1.10)

# 結果を10件表示します。
df_product[['product_cd', 'unit_price', 'price_with_tax']].head(10)

---
> P-069: レシート明細データ（df_receipt）と商品データ（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果を10件表示せよ。

In [ ]:
# df_receiptとdf_productを結合します。
df_merged = pd.merge(df_receipt, df_product, on='product_cd', how='inner')

# 顧客ごとの全商品の売上金額合計を計算します。
total_sales_by_customer = df_merged.groupby('customer_id')['amount'].sum().reset_index()
total_sales_by_customer.rename(columns={'amount': 'total_amount'}, inplace=True)

# カテゴリ大区分コードが"07"（瓶詰缶詰）の売上金額合計を計算します。
category07_sales_by_customer = df_merged[df_merged['category_major_cd'] == '07'].groupby('customer_id')['amount'].sum().reset_index()
category07_sales_by_customer.rename(columns={'amount': 'category07_amount'}, inplace=True)

# 両方のデータを結合します。
df_result = pd.merge(total_sales_by_customer, category07_sales_by_customer, on='customer_id', how='inner')

# 比率を計算します。
df_result['ratio'] = df_result['category07_amount'] / df_result['total_amount']
df_result.head(10)

---
> P-070: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [ ]:
# sales_ymdを日付型に変換します。
df_receipt['sales_ymd'] = pd.to_datetime(df_receipt['sales_ymd'].astype(str))

# application_dateを日付型に変換します。
df_customer['application_date'] = pd.to_datetime(df_customer['application_date'])

# df_receiptとdf_customerをcustomer_idで結合します。
df_merged = pd.merge(df_receipt, df_customer[['customer_id', 'application_date']], on='customer_id', how='inner')

# 経過日数を計算します。
df_merged['elapsed_days'] = (df_merged['sales_ymd'] - df_merged['application_date']).dt.days
df_merged[['customer_id', 'sales_ymd', 'application_date', 'elapsed_days']].head(10)

---
> P-071: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [ ]:
# sales_ymdを日付型に変換します。
df_receipt['sales_ymd'] = pd.to_datetime(df_receipt['sales_ymd'].astype(str))

# application_dateを日付型に変換します。
df_customer['application_date'] = pd.to_datetime(df_customer['application_date'])

# df_receiptとdf_customerをcustomer_idで結合します。
df_merged = pd.merge(df_receipt, df_customer[['customer_id', 'application_date']], on='customer_id', how='inner')

# 経過月数を計算します。
# relativedeltaを使用して月数を計算し、1ヶ月未満は切り捨てます。
df_merged['elapsed_months'] = df_merged.apply(lambda row: relativedelta(row['sales_ymd'], row['application_date']).years * 12 + relativedelta(row['sales_ymd'], row['application_date']).months, axis=1)
df_merged[['customer_id', 'sales_ymd', 'application_date', 'elapsed_months']].head(10)

---
> P-072: レシート明細データ（df_receipt）の売上日（df_customer）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [ ]:
# sales_ymdを日付型に変換します。
df_receipt['sales_ymd'] = pd.to_datetime(df_receipt['sales_ymd'].astype(str))

# application_dateを日付型に変換します。
df_customer['application_date'] = pd.to_datetime(df_customer['application_date'])

# df_receiptとdf_customerをcustomer_idで結合します。
df_merged = pd.merge(df_receipt, df_customer[['customer_id', 'application_date']], on='customer_id', how='inner')

# 経過年数を計算します。
# relativedeltaを使用して年数を計算し、1年未満は切り捨てます。
df_merged['elapsed_years'] = df_merged.apply(lambda row: relativedelta(row['sales_ymd'], row['application_date']).years, axis=1)
df_merged[['customer_id', 'sales_ymd', 'application_date', 'elapsed_years']].head(10)

---
> P-073: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [ ]:
# sales_ymdを日付型に変換します。
df_receipt['sales_ymd'] = pd.to_datetime(df_receipt['sales_ymd'].astype(str))

# application_dateを日付型に変換します。
df_customer['application_date'] = pd.to_datetime(df_customer['application_date'])

# df_receiptとdf_customerをcustomer_idで結合します。
df_merged = pd.merge(df_receipt, df_customer[['customer_id', 'application_date']], on='customer_id', how='inner')

# エポック秒による経過時間を計算します。
# .astype('int64') // 10**9 でエポック秒に変換し、差分を計算します。
df_merged['elapsed_epoch_seconds'] = (df_merged['sales_ymd'].astype('int64') // 10**9) - (df_merged['application_date'].astype('int64') // 10**9)
df_merged[['customer_id', 'sales_ymd', 'application_date', 'elapsed_epoch_seconds']].head(10)

---
> P-074: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、直前の月曜日付とともに10件表示せよ（sales_ymdは数値でデータを保持している点に注意）。

In [ ]:
# sales_ymdを日付型に変換します。
df_receipt['sales_ymd'] = pd.to_datetime(df_receipt['sales_ymd'].astype(str))

# 各sales_ymdの週の月曜日を計算します。
# .dt.to_period('W').start_timeで週の開始日（月曜日）を取得できます。
df_receipt['monday_of_week'] = df_receipt['sales_ymd'].dt.to_period('W').apply(lambda r: r.start_time)

# 月曜日からの経過日数を計算します。
df_receipt['days_from_monday'] = (df_receipt['sales_ymd'] - df_receipt['monday_of_week']).dt.days
df_receipt[['sales_ymd', 'monday_of_week', 'days_from_monday']].head(10)

---
> P-075: 顧客データ（df_customer）からランダムに1%のデータを抽出し、先頭から10件表示せよ。

In [ ]:
# df_customerからランダムに1%のデータを抽出します。
# .sample(frac=0.01)を使用します。
# その後、先頭10行を表示します。
df_customer_sampled = df_customer.sample(frac=0.01, random_state=42) # random_stateで再現性を確保
df_customer_sampled.head(10)

---
> P-076: 顧客データ（df_customer）から性別コード（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、性別コードごとに件数を集計せよ。

In [ ]:
# df_customerから性別コード（gender_cd）の割合に基づきランダムに10%のデータを層化抽出します。
# .groupby()と.sample()を組み合わせて使用します。
# random_stateで再現性を確保します。
df_customer_stratified_sampled = df_customer.groupby('gender_cd', group_keys=False).apply(lambda x: x.sample(frac=0.1, random_state=42))

# 性別コードごとに件数を集計します。
df_customer_stratified_sampled['gender_cd'].value_counts()

---
> P-077: レシート明細データ（df_receipt）の売上金額を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。なお、外れ値は売上金額合計を対数化したうえで平均と標準偏差を計算し、その平均から3σを超えて離れたものとする（自然対数と常用対数のどちらでも可）。結果は10件表示せよ。

In [ ]:
# 顧客ごとの売上金額合計を計算します。
customer_amount_sum = df_receipt.groupby('customer_id')['amount'].sum().reset_index()

# 売上金額合計を自然対数化します。
customer_amount_sum['amount_log'] = np.log(customer_amount_sum['amount'])

# 対数化した売上金額の平均と標準偏差を計算します。
mean_log_amount = customer_amount_sum['amount_log'].mean()
std_log_amount = customer_amount_sum['amount_log'].std()

# 外れ値を抽出します（平均から3σを超えて離れたもの）。
# 絶対値を使用することで、平均から上にも下にも3σ離れたものを検出します。
outliers = customer_amount_sum[np.abs(customer_amount_sum['amount_log'] - mean_log_amount) > 3 * std_log_amount]
outliers.head(10)

---
> P-078: レシート明細データ（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第1四分位と第3四分位の差であるIQRを用いて、「第1四分位数-1.5×IQR」を下回るもの、または「第3四分位数+1.5×IQR」を超えるものとする。結果は10件表示せよ。

In [ ]:
# df_receiptからcustomer_idが"Z"で始まらない行をフィルタリングします。
# customer_idごとにamountを合計します。
df_receipt_filtered = df_receipt[~df_receipt['customer_id'].str.startswith('Z')]
customer_amount_sum = df_receipt_filtered.groupby('customer_id')['amount'].sum().reset_index()

# 四分位点とIQRを計算します。
Q1 = customer_amount_sum['amount'].quantile(0.25)
Q3 = customer_amount_sum['amount'].quantile(0.75)
IQR = Q3 - Q1

# 外れ値の閾値を計算します。
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 外れ値を抽出します。
outliers = customer_amount_sum[(customer_amount_sum['amount'] < lower_bound) | (customer_amount_sum['amount'] > upper_bound)]
outliers.head(10)

---
> P-079: 商品データ（df_product）の各項目に対し、欠損数を確認せよ。

In [ ]:
# df_productの各列の欠損値の数をカウントします。
# .isnull()で欠損値のDataFrameを作成し、.sum()で列ごとの合計を計算します。
df_product.isnull().sum()

---
> P-080: 商品データ（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たな商品データを作成せよ。なお、削除前後の件数を表示させ、079で確認した件数だけ減少していることも確認すること。

In [ ]:
# 欠損値削除前の行数を取得します。
original_rows = len(df_product)

# いずれかの項目に欠損があるレコードを全て削除します。
# .dropna()メソッドを使用します。
df_product_cleaned = df_product.dropna()

# 欠損値削除後の行数を取得します。
cleaned_rows = len(df_product_cleaned)

# 削除前後の件数と減少数を確認します。
print(f"削除前の行数: {original_rows}")
print(f"削除後の行数: {cleaned_rows}")
print(f"減少した行数: {original_rows - cleaned_rows}")

---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たな商品データを作成せよ。なお、平均値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
# unit_priceとunit_costの平均値を計算し、1円未満を丸めます。
mean_unit_price = round(df_product['unit_price'].mean())
mean_unit_cost = round(df_product['unit_cost'].mean())

# 欠損値を平均値で補完します。
# .fillna()メソッドを使用します。
df_product_filled = df_product.copy()
df_product_filled['unit_price'].fillna(mean_unit_price, inplace=True)
df_product_filled['unit_cost'].fillna(mean_unit_cost, inplace=True)

# 欠損値がないことを確認します。
df_product_filled.isnull().sum()

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
# unit_priceとunit_costの中央値を計算し、1円未満を丸めます。
median_unit_price = round(df_product['unit_price'].median())
median_unit_cost = round(df_product['unit_cost'].median())

# 欠損値を中央値で補完します。
df_product_filled_median = df_product.copy()
df_product_filled_median['unit_price'].fillna(median_unit_price, inplace=True)
df_product_filled_median['unit_cost'].fillna(median_unit_cost, inplace=True)

# 欠損値がないことを確認します。
df_product_filled_median.isnull().sum()

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品のカテゴリ小区分コード（category_small_cd）ごとに算出した中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [ ]:
# category_small_cdごとにunit_priceとunit_costの中央値を計算し、1円未満を丸めます。
df_product_filled_group = df_product.copy()
df_product_filled_group['unit_price'] = df_product_filled_group.groupby('category_small_cd')['unit_price'].transform(lambda x: x.fillna(round(x.median())))
df_product_filled_group['unit_cost'] = df_product_filled_group.groupby('category_small_cd')['unit_cost'].transform(lambda x: x.fillna(round(x.median())))

# 欠損値がないことを確認します。
df_product_filled_group.isnull().sum()

---
> P-084: 顧客データ（df_customer）の全顧客に対して全期間の売上金額に占める2019年売上金額の割合を計算し、新たなデータを作成せよ。ただし、売上実績がない場合は0として扱うこと。そして計算した割合が0超のものを抽出し、結果を10件表示せよ。また、作成したデータに欠損が存在しないことを確認せよ。

In [ ]:
# sales_ymdを日付型に変換します。
df_receipt['sales_ymd'] = pd.to_datetime(df_receipt['sales_ymd'].astype(str))

# 顧客ごとの全期間の売上金額合計を計算します。
total_sales = df_receipt.groupby('customer_id')['amount'].sum().reset_index()
total_sales.rename(columns={'amount': 'total_amount'}, inplace=True)

# 2019年の売上金額を抽出します。
df_receipt_2019 = df_receipt[df_receipt['sales_ymd'].dt.year == 2019]
sales_2019 = df_receipt_2019.groupby('customer_id')['amount'].sum().reset_index()
sales_2019.rename(columns={'amount': 'amount_2019'}, inplace=True)

# 全顧客データと結合し、売上実績がない場合は0で埋めます。
df_customer_sales_ratio = pd.merge(df_customer[['customer_id']], total_sales, on='customer_id', how='left').fillna(0)
df_customer_sales_ratio = pd.merge(df_customer_sales_ratio, sales_2019, on='customer_id', how='left').fillna(0)

# 2019年売上金額の割合を計算します。
# total_amountが0の場合は0とします。
df_customer_sales_ratio['sales_ratio_2019'] = df_customer_sales_ratio.apply(lambda row: row['amount_2019'] / row['total_amount'] if row['total_amount'] != 0 else 0, axis=1)

# 割合が0超のものを抽出し、先頭10件表示します。
df_result = df_customer_sales_ratio[df_customer_sales_ratio['sales_ratio_2019'] > 0]
df_result.head(10)

# 欠損値がないことを確認します。
df_result.isnull().sum()

---
> P-085: 顧客データ（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いてジオコードデータ（df_geocode）を紐付け、新たな顧客データを作成せよ。ただし、1つの郵便番号（postal_cd）に複数の経度（longitude）、緯度（latitude）情報が紐づく場合は、経度（longitude）、緯度（latitude）の平均値を算出して使用すること。また、作成結果を確認するために結果を10件表示せよ。

In [ ]:
# df_geocodeでpostal_cdごとに経度と緯度の平均値を計算します。
df_geocode_agg = df_geocode.groupby('postal_cd')[['longitude', 'latitude']].mean().reset_index()

# df_customerとdf_geocode_aggをpostal_cdで結合します。
df_customer_geo = pd.merge(df_customer, df_geocode_agg, on='postal_cd', how='left')
df_customer_geo.head(10)

---
> P-086: 085で作成した緯度経度つき顧客データに対し、会員申込店舗コード（application_store_cd）をキーに店舗データ（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客住所（address）の緯度・経度を用いて申込み店舗と顧客住所の距離（単位：km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は以下の簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示せよ。

$$
\mbox{緯度（ラジアン）}：\phi \\
\mbox{経度（ラジアン）}：\lambda \\
\mbox{距離}L = 6371 * \arccos(\sin \phi_1 * \sin \phi_2
+ \cos \phi_1 * \cos \phi_2 * \cos(\lambda_1 − \lambda_2))
$$

In [ ]:
# df_customer_geo（P-085で作成）とdf_storeをapplication_store_cdで結合します。
df_merged = pd.merge(df_customer_geo, df_store, left_on='application_store_cd', right_on='store_cd', how='left', suffixes=('_customer', '_store'))

# 距離計算関数を定義します。
# 簡易的な計算式を使用します。
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # 地球の半径 (km)
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)

    delta_lon = lon2_rad - lon1_rad
    
    # 緯度（ラジアン）：phi
    # 経度（ラジアン）：lambda
    # 距離L = 6371 * arccos(sin phi1 * sin phi2 + cos phi1 * cos phi2 * cos(lambda1 - lambda2))
    distance = R * np.arccos(np.sin(lat1_rad) * np.sin(lat2_rad) +
                             np.cos(lat1_rad) * np.cos(lat2_rad) * np.cos(delta_lon))
    return distance

# 距離を計算します。
df_merged['distance_km'] = df_merged.apply(lambda row: calculate_distance(row['latitude_customer'], row['longitude_customer'], row['latitude_store'], row['longitude_store']), axis=1)

# 結果を10件表示します。
df_merged[['customer_id', 'address_customer', 'address_store', 'distance_km']].head(10)

---
> P-087: 顧客データ（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなして1顧客1レコードとなるように名寄せした名寄顧客データを作成し、顧客データの件数、名寄顧客データの件数、重複数を算出せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残し、売上金額合計が同一もしくは売上実績がない顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [ ]:
# df_receiptとdf_customerをcustomer_idで結合し、顧客ごとの売上金額合計を計算します。
df_customer_sales = pd.merge(df_customer, df_receipt.groupby('customer_id')['amount'].sum().reset_index(), on='customer_id', how='left')
df_customer_sales['amount'].fillna(0, inplace=True) # 売上実績がない顧客は0で埋める

# customer_nameとpostal_cdでグループ化し、売上金額合計が最も高いものを残します。
# 売上金額合計が同じ場合はcustomer_idが小さいものを残します。
df_customer_deduplicated = df_customer_sales.sort_values(by=['amount', 'customer_id'], ascending=[False, True]).drop_duplicates(subset=['customer_name', 'postal_cd'])

# 顧客データの件数、名寄顧客データの件数、重複数を算出します。
original_customer_count = len(df_customer)
deduplicated_customer_count = len(df_customer_deduplicated)
duplicate_count = original_customer_count - deduplicated_customer_count

print(f"顧客データの件数: {original_customer_count}")
print(f"名寄顧客データの件数: {deduplicated_customer_count}")
print(f"重複数: {duplicate_count}")

---
> P-088: 087で作成したデータを元に、顧客データに統合名寄IDを付与したデータを作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定
> 
> 顧客IDのユニーク件数と、統合名寄IDのユニーク件数の差も確認すること。

In [ ]:
# df_customer_sales（P-087で作成）を基に、統合名寄IDを付与します。
# customer_nameとpostal_cdでグループ化し、各グループの最初のcustomer_idを統合名寄IDとします。
df_customer_sales['consolidated_id'] = df_customer_sales.groupby(['customer_name', 'postal_cd'])['customer_id'].transform('min')

# 顧客IDのユニーク件数と、統合名寄IDのユニーク件数を計算します。
unique_customer_ids = df_customer_sales['customer_id'].nunique()
unique_consolidated_ids = df_customer_sales['consolidated_id'].nunique()
difference = unique_customer_ids - unique_consolidated_ids

print(f"顧客IDのユニーク件数: {unique_customer_ids}")
print(f"統合名寄IDのユニーク件数: {unique_consolidated_ids}")
print(f"差: {difference}")

---
> P-089: 売上実績がある顧客を、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [ ]:
# 売上実績がある顧客を抽出します。
customers_with_sales = df_receipt['customer_id'].unique()
df_customers_with_sales = df_customer[df_customer['customer_id'].isin(customers_with_sales)]

# 学習用データとテスト用データに8:2の割合で分割します。
# sklearn.model_selectionのtrain_test_splitを使用します。
# random_stateで再現性を確保します。
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df_customers_with_sales, test_size=0.2, random_state=42)

print(f"学習用データの件数: {len(train_data)}")
print(f"テスト用データの件数: {len(test_data)}")

---
> P-090: レシート明細データ（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月の時系列モデル構築用データを3セット作成せよ。

In [ ]:
# sales_ymdを日付型に変換します。
df_receipt['sales_ymd'] = pd.to_datetime(df_receipt['sales_ymd'].astype(str))

# 月次で売上金額を集計します。
df_monthly_sales = df_receipt.set_index('sales_ymd').resample('M')['amount'].sum().reset_index()

# 時系列モデル構築用データを3セット作成します。
# 各セットで学習用12ヶ月、テスト用6ヶ月とします。
# データの期間は2017年1月1日〜2019年10月31日なので、
# 2017-01から2019-10までの34ヶ月分のデータがあります。
# 18ヶ月のセットを3つ作成します。

# セット1: 2017-01から2017-12 (学習), 2018-01から2018-06 (テスト)
train_set1 = df_monthly_sales[(df_monthly_sales['sales_ymd'] >= '2017-01-01') & (df_monthly_sales['sales_ymd'] <= '2017-12-31')]
test_set1 = df_monthly_sales[(df_monthly_sales['sales_ymd'] >= '2018-01-01') & (df_monthly_sales['sales_ymd'] <= '2018-06-30')]

# セット2: 2017-07から2018-06 (学習), 2018-07から2018-12 (テスト)
train_set2 = df_monthly_sales[(df_monthly_sales['sales_ymd'] >= '2017-07-01') & (df_monthly_sales['sales_ymd'] <= '2018-06-30')]
test_set2 = df_monthly_sales[(df_monthly_sales['sales_ymd'] >= '2018-07-01') & (df_monthly_sales['sales_ymd'] <= '2018-12-31')]

# セット3: 2018-01から2018-12 (学習), 2019-01から2019-06 (テスト)
train_set3 = df_monthly_sales[(df_monthly_sales['sales_ymd'] >= '2018-01-01') & (df_monthly_sales['sales_ymd'] <= '2018-12-31')]
test_set3 = df_monthly_sales[(df_monthly_sales['sales_ymd'] >= '2019-01-01') & (df_monthly_sales['sales_ymd'] <= '2019-06-30')]

print("セット1 - 学習データ期間:", train_set1['sales_ymd'].min(), "to", train_set1['sales_ymd'].max())
print("セット1 - テストデータ期間:", test_set1['sales_ymd'].min(), "to", test_set1['sales_ymd'].max())
print("セット2 - 学習データ期間:", train_set2['sales_ymd'].min(), "to", train_set2['sales_ymd'].max())
print("セット2 - テストデータ期間:", test_set2['sales_ymd'].min(), "to", test_set2['sales_ymd'].max())
print("セット3 - 学習データ期間:", train_set3['sales_ymd'].min(), "to", train_set3['sales_ymd'].max())
print("セット3 - テストデータ期間:", test_set3['sales_ymd'].min(), "to", test_set3['sales_ymd'].max())

---
> P-091: 顧客データ（df_customer）の各顧客に対し、売上実績がある顧客数と売上実績がない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [ ]:
# 売上実績がある顧客とない顧客を特定します。
customers_with_sales = df_receipt['customer_id'].unique()
df_customer['has_sales'] = df_customer['customer_id'].isin(customers_with_sales)

# 売上実績がある顧客とない顧客に分割します。
df_has_sales = df_customer[df_customer['has_sales'] == True]
df_no_sales = df_customer[df_customer['has_sales'] == False]

# 少ない方の顧客数に合わせるため、多い方の顧客数をアンダーサンプリングします。
if len(df_has_sales) > len(df_no_sales):
    df_has_sales_sampled = df_has_sales.sample(n=len(df_no_sales), random_state=42)
    df_undersampled = pd.concat([df_has_sales_sampled, df_no_sales])
else:
    df_no_sales_sampled = df_no_sales.sample(n=len(df_has_sales), random_state=42)
    df_undersampled = pd.concat([df_has_sales, df_no_sales_sampled])

print(f"アンダーサンプリング後の売上実績がある顧客数: {df_undersampled['has_sales'].sum()}")
print(f"アンダーサンプリング後の売上実績がない顧客数: {len(df_undersampled) - df_undersampled['has_sales'].sum()}")

---
> P-092: 顧客データ（df_customer）の性別について、第三正規形へと正規化せよ。

In [ ]:
# 性別コード（gender_cd）を第三正規形に正規化します。
# 新しいDataFrameを作成し、gender_cdとgender_nameをマッピングします。
# gender_nameは0:男性, 1:女性, 9:不明とします。
df_gender = pd.DataFrame({
    'gender_cd': ['0', '1', '9'],
    'gender_name': ['男性', '女性', '不明']
})

# df_customerからgender_cd列を削除し、df_genderと結合することで正規化します。
df_customer_normalized = pd.merge(df_customer.drop('gender_cd', axis=1), df_gender, on='gender_cd', how='left')
df_customer_normalized.head()

---
> P-093: 商品データ（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータ（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データを作成せよ。

In [ ]:
# df_productとdf_categoryを結合し、カテゴリ名を商品データに紐付けます。
# category_major_cd, category_medium_cd, category_small_cdをキーに結合します。
df_product_denormalized = pd.merge(df_product, df_category, on=['category_major_cd', 'category_medium_cd', 'category_small_cd'], how='left')
df_product_denormalized.head()

---
> P-094: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

In [ ]:
# df_product_denormalized（P-093で作成）をCSV形式で出力します。
# index=Falseでインデックスを書き込まないようにします。
# encoding='utf-8'でUTF-8エンコーディングを指定します。
output_path = '../data/product_with_category.csv'
df_product_denormalized.to_csv(output_path, index=False, encoding='utf-8')
print(f"ファイルが {output_path} に出力されました。")

---
> P-095: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|CP932|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

In [ ]:
# df_product_denormalized（P-093で作成）をCSV形式で出力します。
# index=Falseでインデックスを書き込まないようにします。
# encoding='cp932'でCP932エンコーディングを指定します。
output_path = '../data/product_with_category_cp932.csv'
df_product_denormalized.to_csv(output_path, index=False, encoding='cp932')
print(f"ファイルが {output_path} に出力されました。")

---
> P-096: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|無し|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

In [ ]:
# df_product_denormalized（P-093で作成）をCSV形式で出力します。
# header=Falseでヘッダを書き込まないようにします。
# index=Falseでインデックスを書き込まないようにします。
# encoding='utf-8'でUTF-8エンコーディングを指定します。
output_path = '../data/product_with_category_no_header.csv'
df_product_denormalized.to_csv(output_path, index=False, header=False, encoding='utf-8')
print(f"ファイルが {output_path} に出力されました。")

---
> P-097: 094で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|

In [ ]:
# P-094で出力したCSVファイルを読み込みます。
# header=0でヘッダ行を読み込みます。
# encoding='utf-8'でUTF-8エンコーディングを指定します。
input_path = '../data/product_with_category.csv'
df_loaded = pd.read_csv(input_path, encoding='utf-8', header=0)
df_loaded.head(3)

---
> P-098: 096で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|ヘッダ無し|UTF-8|

In [ ]:
# P-096で出力したCSVファイルを読み込みます。
# header=Noneでヘッダ行がないことを指定します。
# encoding='utf-8'でUTF-8エンコーディングを指定します。
input_path = '../data/product_with_category_no_header.csv'
df_loaded_no_header = pd.read_csv(input_path, encoding='utf-8', header=None)
df_loaded_no_header.head(3)

---
> P-099: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

In [ ]:
# P-099: JSONファイルを読み込み、データフレームとして表示します。
# encoding='utf-8'でUTF-8エンコーディングを指定します。
input_path = '../data/product.json'
df_json = pd.read_json(input_path, encoding='utf-8')
df_json.head(3)

---
> P-100: 099で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|

In [ ]:
# P-100: XMLファイルを読み込み、データフレームとして表示します。
# XMLファイルはlxmlエンジンを使用して読み込みます。
input_path = '../data/product.xml'
df_xml = pd.read_xml(input_path)
df_xml.head(3)

# これで１００本終わりです。おつかれさまでした！